In [24]:
import seaborn as sns
sns.set_style("darkgrid")
from datetime import datetime
import numpy as np
import pandas as pd
import tkinter as tk
import time
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from pandastable.core import Table
matplotlib.use('TkAgg')

In [25]:
class nan_page(tk.Frame):
    def __init__(self,root,data):
        self.data=data
        super().__init__(root)
        disp=tk.StringVar()
        disp.set("Checking nan value......")
        tk.Label(self,textvariable=disp).grid(row=0)
        self.baddata=[]
        r,c=data.shape
        for i in range(r):
            if True in data.iloc[i,:].isnull().values:
                self.baddata.append(i)
        disp.set("Result:")
        if len(self.baddata)==0:
            tk.Label(self,text='Great! The data does not contain any nan entry~').grid(row=1)
        else:
            tk.Label(self,text='Hey, rows containing nan values are listed below.').grid(row=1)
            self.frame=tk.Frame(self)
            self.table=Table(self.frame,dataframe=data.iloc[self.baddata,:])
            self.table.show()
            self.frame.grid(row=2)
            tk.Label(self,text="How do you want to fix those nan values?").grid(row=3)
            tk.Button(self,text="delete",command=self.ondelete).grid(row=4,column=0)
            tk.Button(self,text="set to 0",command=self.setzero).grid(row=4,column=1)
            tk.Button(self,text="forward fill",command=self.ffill).grid(row=4,column=2)
            tk.Button(self,text="linear interpolate",command=self.interpolate).grid(row=4,column=3)

    def ondelete(self):
        self.data.dropna(inplace=True)
        self.table.model.df=self.data
        self.table.redraw()
    def setzero(self):
        self.data.fillna(0,inplace=True)
        self.table.model.df=self.data
        self.table.redraw()
    def ffill(self):
        self.data.fillna(method='ffill',inplace=True)
        self.table.model.df=self.data
        self.table.redraw()
    def interpolate(self):
        self.data.interpolate(axis=0,inplace=True)
        self.table.model.df=self.data
        self.table.redraw()

In [26]:
class dist_page(tk.Frame):
    def __init__(self,root,data):
        super().__init__(root)
        self.data=data
        tk.Label(self,text="Choose the column you would like to plot a distribution for: ").pack(side="top")
        option=list(data.columns)
        self.v=tk.StringVar()
        self.v.set("please select the column")
        tk.OptionMenu(root,self.v,*option,command=self.onselection).pack(side="top")
        f =Figure(figsize=(5,4), dpi=100)
        self.a = f.add_subplot(111)
        self.canvas =FigureCanvasTkAgg(f, master=self)
        self.canvas._tkcanvas.pack(side="top", fill="both", expand=True)
    def onselection(self,col):
        self.a.clear()
        sns.hist(x=self.data.index,y=self.data.loc[:,col],ax=self.a)
        self.a.hist(self.data.loc[:,col])
        self.canvas.draw()
        

In [27]:
class trend_page(tk.Frame):
    def __init__(self,root,data):
        super().__init__(root)
        self.data=data
        tk.Label(self,text="Choose the column you would like to plot the trend for: ").pack(side="top")
        option=list(data.columns)
        self.v=tk.StringVar()
        self.v.set("please select the column")
        tk.OptionMenu(root,self.v,*option,command=self.onselection).pack(side="top")
        f =Figure(figsize=(5,4), dpi=100)
        self.a = f.add_subplot(111)
        self.canvas =FigureCanvasTkAgg(f, master=self)
        self.canvas._tkcanvas.pack(side="top", fill="both", expand=True)
    def onselection(self,col):
        self.a.clear()
        sns.lineplot(x=self.data.index,y=self.data.loc[:,col],ax=self.a)
        self.canvas.draw()
        

In [28]:
class corr_page(tk.Frame):
    def __init__(self,root,data):
        super().__init__(root)
        self.data=data
        tk.Label(self,text="Choose the columns you would like to plot the correlation heatmap for: ").pack(side="top")
        self.option=list(data.columns)
        self.result=[]
        for i in self.option:
            self.result.append(tk.IntVar())
            b = tk.Checkbutton(root,text=i,variable=self.result[-1])
            b.pack(side="left")
        bt=tk.Button(self,text='Draw',command=self.onselection)
        bt.pack(side='top')
        f = Figure()
        self.a = f.add_subplot(111)
        self.canvas =FigureCanvasTkAgg(f, master=self)
        self.canvas._tkcanvas.pack(side="top", fill="both", expand=True)
    def onselection(self,event=None):
        n=len(self.option)
        result = [self.option[i] for i in range(n) if self.result[i].get()]
        hm=np.corrcoef(self.data.loc[:,result],rowvar=False)
        self.a.clear()
        sns.heatmap(hm,ax=self.a,annot=True,vmin=-1, vmax=1,cmap="YlGnBu",cbar=False,xticklabels=result, yticklabels=result)
        self.canvas.draw()